In [250]:
import numpy as np
import datajoint as dj
import time
import pymeshfix
import os
import datetime
import calcification_Module as cm

#for supressing the output
import os, contextlib
import pathlib
import subprocess

#for error counting
from collections import Counter

#for reading in the new raw_skeleton files
import csv

from meshparty import trimesh_io

#for filtering
import math
from pykdtree.kdtree import KDTree


ImportError: cannot import name 'rtree'

In [2]:
#setting the address and the username
dj.config['database.host'] = '10.28.0.34'
dj.config['database.user'] = 'celiib'
dj.config['database.password'] = 'newceliipass'
dj.config['safemode']=True
dj.config["display.limit"] = 20

schema = dj.schema('microns_ta3p100')
ta3p100 = dj.create_virtual_module('ta3p100', 'microns_ta3p100')


Connecting celiib@10.28.0.34:3306


In [31]:
#create the output file
##write the OFF file for the neuron
import pathlib
def write_Whole_Neuron_Off_file(neuron_ID,
                                vertices=[], 
                                triangles=[],
                                folder="pymesh_neurons"):
    #primary_key = dict(segmentation=1, segment_id=segment_id, decimation_ratio=0.35)
    #vertices, triangles = (mesh_Table_35 & primary_key).fetch1('vertices', 'triangles')
    
    num_vertices = (len(vertices))
    num_faces = len(triangles)
    
    #get the current file location
    file_loc = pathlib.Path.cwd() / folder
    filename = "neuron_" + str(neuron_ID)
    path_and_filename = file_loc / filename
    
    #print(file_loc)
    #print(path_and_filename)
    
    #open the file and start writing to it    
    f = open(str(path_and_filename) + ".off", "w")
    f.write("OFF\n")
    f.write(str(num_vertices) + " " + str(num_faces) + " 0\n" )
    
    
    #iterate through and write all of the vertices in the file
    for verts in vertices:
        f.write(str(verts[0]) + " " + str(verts[1]) + " " + str(verts[2])+"\n")
    
    #print("Done writing verts")
        
    for faces in triangles:
        f.write("3 " + str(faces[0]) + " " + str(faces[1]) + " " + str(faces[2])+"\n")
    
    print("Done writing OFF file")
    #f.write("end")
    
    return str(path_and_filename),str(filename),str(file_loc)

In [29]:
import numpy as np

def index_unique_rows(full_coordinate_array):
    """
    Separates an array of nested coordinate rows into an array of unique rows and and index array.
    """
    vertices, flat_idx = np.unique(full_coordinate_array.reshape(-1, full_coordinate_array.shape[-1]), axis=0, return_inverse=True)
    return vertices, flat_idx.reshape(-1, full_coordinate_array.shape[-2])





In [82]:
#testing the soma extraction

segment_id = 648518346349475523

segment_id = 648518346349481014 #one we were testing earlier

neuron_axon = 648518346349507130
neuron_target = 648518346341393609

segment_id = neuron_target

segmentation = 2
query_key=dict(segment_id=segment_id,segmentation=segmentation)

soma_table = (ta3p100.CompartmentFinal.ComponentFinal() & "compartment_type='Soma'" )
vertices_soma,triangles_soma = (soma_table & query_key).fetch("vertex_indices","triangle_indices")
ts_flatten = np.hstack(triangles_soma)

#get the cleansed mesh triangles
vertices_mesh,triangles_mesh = (ta3p100.CleansedMesh & query_key).fetch("vertices","triangles")

vertices_real = vertices_mesh[0]
triangles_real = triangles_mesh[0]

ts_stack_whole = vertices_real[triangles_real[ts_flatten]]

vertices_whole, triangles_whole = index_unique_rows(ts_stack_whole)

In [48]:
write_Whole_Neuron_Off_file(str(segment_id)+ "whole",vertices_whole,triangles_whole)

Done writing OFF file


('/notebooks/14_Contacts/pymesh_neurons/neuron_648518346349481014whole',
 'neuron_648518346349481014whole',
 '/notebooks/14_Contacts/pymesh_neurons')

In [83]:
#get the bounding box by using trimesh
mesh = trimesh_io.Mesh(vertices=vertices_whole, faces=triangles_whole)
mesh.bounding_box.vertices
#mesh.bounding_box.export("./bbox_whole_upright.off","off")

#get the maximum points of the vertices

#two highest points for the bounding box
min_bb = np.array(mesh.bounding_box.vertices).min(0)
max_bb = np.array(mesh.bounding_box.vertices).max(0)

threshold = 3500

min_bb_zone = min_bb + threshold
max_bb_zone = max_bb + threshold

print(min_bb_zone,max_bb_zone)

[365233.3125     268905.40625     12156.08984375] [385254.09375    284683.9375      28259.78320312]


In [90]:
#want to get the edges that still persist on the presynaptic
neuron_axon = 648518346349507130
neuron_target = 648518346341393609

segment_id = neuron_target

segmentation = 2
query_key_presyn=dict(segment_id=segment_id,segmentation=segmentation)

target_edges = (ta3p100.FilteredSkeleton() & query_key_presyn).fetch(as_dict=True)

In [87]:
ta3p100.FilteredSkeleton()

segmentation segmentation id,segment_id segment id unique within each Segmentation,spine_filter_length,n_edges,edges
2,648518346341351467,2500,517,=BLOB=
2,648518346341351503,2500,323,=BLOB=
2,648518346341351508,2500,486,=BLOB=
2,648518346341351509,2500,348,=BLOB=
2,648518346341351512,2500,584,=BLOB=
2,648518346341351514,2500,565,=BLOB=
2,648518346341351518,2500,397,=BLOB=
2,648518346341351523,2500,334,=BLOB=
2,648518346341351525,2500,470,=BLOB=
2,648518346341351531,2500,342,=BLOB=


In [106]:
ta3p100.NeuronRawSkeleton()

segmentation segmentation id,segment_id segment id unique within each Segmentation,time_updated the time at which the component labels were updated,n_edges number of edges stored,edges array storing edges on each row,n_vertices number of vertices in mesh filtered of error segments,n_triangles number of faces in mesh filtered of error segments,vertices mesh data for vertices in mesh filtered of error segments,triangles mesh data for faces in mesh filtered of error segments
2,648518346341352006,2019-03-29 18:55:10,1420,=BLOB=,25455,50819,=BLOB=,=BLOB=
2,648518346341352223,2019-03-29 18:55:22,1080,=BLOB=,39026,77823,=BLOB=,=BLOB=
2,648518346341353019,2019-03-29 18:55:25,722,=BLOB=,8805,17592,=BLOB=,=BLOB=
2,648518346341353186,2019-03-29 18:55:40,1220,=BLOB=,43636,86901,=BLOB=,=BLOB=
2,648518346341353521,2019-03-29 18:55:41,0,=BLOB=,0,0,=BLOB=,=BLOB=
2,648518346341353574,2019-03-29 18:55:58,1598,=BLOB=,56775,113343,=BLOB=,=BLOB=
2,648518346341353607,2019-03-29 18:56:19,1152,=BLOB=,68234,136186,=BLOB=,=BLOB=
2,648518346341353788,2019-03-29 18:56:25,831,=BLOB=,9789,19506,=BLOB=,=BLOB=
2,648518346341353883,2019-03-29 18:56:39,1808,=BLOB=,38760,77338,=BLOB=,=BLOB=
2,648518346341353986,2019-03-29 18:56:29,0,=BLOB=,0,0,=BLOB=,=BLOB=


In [86]:
ta3p100.NeuriteRawSkeleton & query_key_presyn

segmentation segmentation id,segment_id segment id unique within each Segmentation,time_updated the time at which the skeleton was generated,n_edges number of edges stored,edges array storing edges on each row,n_bodies the amount of segments the neurite was originally split into,lagest_mesh_perc the percentage of the entire mesh that the largest submesh makes up
2,648518346349507130,2019-03-29 23:45:06,316,=BLOB=,21,0.404139


In [95]:
target_edges[0]["edges"]

array([[[390503. , 258004. ,  53092. ],
        [390297. , 258083. ,  53343.4]],

       [[360268. , 247349. ,  34906.8],
        [362103. , 247974. ,  35768.2]],

       [[371456. , 298452. ,  23197.6],
        [371367. , 297628. ,  23062.2]],

       ...,

       [[372090. , 279077. ,  18538.1],
        [371983. , 280015. ,  18723.2]],

       [[372090. , 279077. ,  18538.1],
        [372325. , 276610. ,  18883. ]],

       [[373011. , 275048. ,  16765. ],
        [372325. , 276610. ,  18883. ]]])

In [113]:
import time
start_time = time.time()
edges = target_edges[0]["edges"]
filtered_edges = list()
for i,e in enumerate(a):
    
#     print(min(e[0][0],e[1][0]))
#     print(max_bb_zone[0])
#     print(min(e[0][0],e[1][0])>max_bb_zone[0])
    
    if min(e[0][0],e[1][0])>max_bb_zone[0]:
        continue
    
    if max(e[0][0],e[1][0])<min_bb_zone[0]:
        continue
    
    if min(e[0][1],e[1][1])>max_bb_zone[1]:
        continue
    
    if max(e[0][1],e[1][1])<min_bb_zone[1]:
        continue
        
    if min(e[0][2],e[1][2])>max_bb_zone[2]:
        continue
    
    if max(e[0][2],e[1][2])<min_bb_zone[2]:
        continue
        
    filtered_edges.append(e)


print(f"Loops took {time.time()-start_time} sec")
     

Loops took 0.009096145629882812 sec


In [114]:
print(len(filtered_edges),len(edges))

52 1326


In [ ]:
#function that will eliminate any edges that aren't in the bounding box
def filter_edges_bounding_box(min_bb_zone,max_bb_zone,edges):
    import time
start_time = time.time()
edges = target_edges[0]["edges"]
filtered_edges = list()
for i,e in enumerate(a):
    
#     print(min(e[0][0],e[1][0]))
#     print(max_bb_zone[0])
#     print(min(e[0][0],e[1][0])>max_bb_zone[0])
    
    if min(e[0][0],e[1][0])>max_bb_zone[0]:
        continue
    
    if max(e[0][0],e[1][0])<min_bb_zone[0]:
        continue
    
    if min(e[0][1],e[1][1])>max_bb_zone[1]:
        continue
    
    if max(e[0][1],e[1][1])<min_bb_zone[1]:
        continue
        
    if min(e[0][2],e[1][2])>max_bb_zone[2]:
        continue
    
    if max(e[0][2],e[1][2])<min_bb_zone[2]:
        continue
        
    filtered_edges.append(e)
    
    return np.array(filtered_edges)

In [118]:
def calculate_distance(p1,p2):
    distance = math.sqrt((p1[0]-p2[0])**2 + (p1[1]-p2[1])**2 + (p1[2]-p2[2])**2)
    return distance

In [122]:
#now sample all of the edges and add them in line
resolution = 50

# #how to divide up a line
# Cx = Ax * (1-t) + Bx * t
# Cy = Ay * (1-t) + By * t

#how to find the number of ways to divide up


#linear algebra one was faster
start_time = time.time()
filtered_edges_distance_lin_alg = [np.linalg.norm(a[0]-a[1]) for a in  filtered_edges]
print(f"lin alg builder took {time.time()-start_time} sec")


start_time = time.time()
filtered_edges_distance = [calculate_distance(a[0],a[1]) for a in filtered_edges]
print(f"function builder took {time.time()-start_time} sec")
#for fe in filtered_edges:
    


lin alg builder took 0.0021209716796875 sec
function builder took 0.00043892860412597656 sec


In [183]:
resolution = 100
print("filtered_edges.shape = " + str(np.array(filtered_edges).shape))
discrete_edges = []
discrete_edges_labels = []

for i,(edg,edg_len) in enumerate(zip(filtered_edges,filtered_edges_distance_lin_alg)):
    #find the number of segments to divide line into
    n_segments = math.ceil(edg_len/resolution)
    discrete_edges.append(np.linspace(edg[0],edg[1],n_segments+1))
    labels_counter = [i]*(n_segments + 1)
    discrete_edges_labels.append(labels_counter)
        
#vertically stack all of the new edges
discrete_edges_stacked = np.vstack(discrete_edges)
discrete_edges_labels_stacked = np.hstack(discrete_edges_labels)
print("discrete_edges.shape = " + str(np.array(discrete_edges).shape))
print("discrete_edges_stacked.shape = " + str(discrete_edges_stacked.shape))
print("discrete_edges_labels.shape = " + str(np.array(discrete_edges_labels).shape))
print("discrete_edges_labels_stacked.shape = " + str(discrete_edges_labels_stacked.shape))
                          
#unravels it so now just one long array with some repeat vertices of the edges

filtered_edges.shape = (52, 2, 3)
discrete_edges.shape = (52,)
discrete_edges_stacked.shape = (878, 3)
discrete_edges_labels.shape = (52,)
discrete_edges_labels_stacked.shape = (878,)


In [195]:
#do KDTree for finding the matching points on the mesh for all the discrete points

kdtree = KDTree(vertices_whole)

distances, nearest_nodes = kdtree.query(discrete_edges_stacked)

boundary_points = [((np.linspace(d,discrete_edges_stacked[i+1],3))[1]) 
                           for i,d in enumerate(discrete_edges_stacked) 
                           if i <(len(discrete_edges_stacked) - 1)]

boundary_points.insert(0,discrete_edges_stacked[0])
boundary_points.append(discrete_edges_stacked[len(discrete_edges_stacked)-1])

boundary_points = np.array(boundary_points)
print(boundary_points.shape)

print(boundary_points)

boundary_distances, boundary_nearest_nodes = kdtree.query(boundary_points)

#print(distances.shape)
#print(distances)
#print(nearest_nodes.shape)
#print(nearest_nodes)
#np.vstack((nearest_nodes,discrete_edges_labels_stacked))

(879, 3)
[[372434.         276916.          22396.9       ]
 [372433.60714286 276949.85714286  22433.18214286]
 [372432.82142857 277017.57142857  22505.74642857]
 ...
 [372361.75       276526.32142857  18769.53571429]
 [372337.25       276582.10714286  18845.17857143]
 [372325.         276610.          18883.        ]]


In [218]:
print(len(boundary_nearest_nodes))
print(len(distances))
print(len(nearest_nodes))
print(len(discrete_edges_labels_stacked))

879
878
878
878


In [229]:
# #filter away the edges that are not within the threshold distance
# """ pseudocode: 
# Go through one edge group (designated by counter) at a time
# 1) If first point is under threshold --> put as "start" else...
# 2) go find first point under threshold and put the average of previous and that one in list "start"
#     c. If never find one then just move onto next list
# 3) Go until you find under threshold: 
#     a. Once do, add the average of previous and that one as stop and add (start,stop to list)
#     b. if never find end then add last point as stop and add (start,stop to list)
# 4) Repeat step 4



# 1) if below the threshold, and new counter, add to list
# 2) if above threshold, and new counter --> ignore and move to next one
# 3) if below the threshold but not a new counter --> ignore and move on
# 4) if above threshold but not a new counter --> add it to the list as endpoint


# """
# # boundary_points = [((np.linspace(d,discrete_edges_stacked[i+1],3))[1]) 
# #                            for i,d in enumerate(discrete_edges_stacked) 
# #                            if i <(len(discrete_edges_stacked) - 1)]

# # boundary_points.insert(0,discrete_edges_stacked[0])
# # boundary_points.append(discrete_edges_stacked[len(discrete_edges_stacked)-1])

# myCounter = Counter(discrete_edges_labels_stacked)
# print(dict(myCounter.items()))

# global_counter = 0
# final_contact_skeleton = []
# for group,number_pts in dict(myCounter).items():
#     #find first point that is under threshold
#     local_counter = global_counter
    
#     found_End = False
#     while(found_End==False):
#         print("group = " + str(group))
#         print("local_counter = " + str(local_counter))
#         print("global_counter = " + str(global_counter))
#         start = np.array([])
#         for i in range(local_counter,global_counter + number_pts):
#             if distances[i]<threshold:
#                 if i == global_counter:
#                     #print(nearest_nodes[i])
#                     start=vertices_whole[nearest_nodes[i]]
#                 else:
#                     start=vertices_whole[boundary_nearest_nodes[i]] 
#                 local_counter = i + 1
#                 break

#         #if got through whole list and not find start point
#         if not start.any():
#             print("went through whole list")
#             global_counter += number_pts
#             found_End = True
#             break
#         else: #if did get a start value 
#             # go until you get an end point 
#             print("local counter = " + str(local_counter))
#             stop = np.array([])
#             for i in range(local_counter,global_counter + number_pts):
#                 if distances[i]>threshold:
#                     stop=vertices_whole[boundary_nearest_nodes[i]]
#                     local_counter = i + 1
#                     print("local counter after stop = " + str(local_counter))
#                     break

#             #if never hit stop point
#             if not stop.any():
#                 print("never hit the stop spot")
#                 #print(nearest_nodes[global_counter + number_pts - 1])
#                 stop = vertices_whole[nearest_nodes[global_counter + number_pts - 1]]
#                 #add the stop and start to the list
#                 print("about to append 1 " + str((start,stop)))
#                 final_contact_skeleton.append((start,stop))
#                 global_counter += number_pts
#                 #go onto next line segment
#                 found_End=True
#                 break
#             else:
#                 #add the stop label
#                 print("about to append 2 " + str((start,stop)))
#                 final_contact_skeleton.append((start,stop))

#             #continue with loop if can find another start and stop
            
# final_contact_skeleton = np.array(final_contact_skeleton)
# print(final_contact_skeleton.shape)
    


{0: 15, 1: 14, 2: 12, 3: 15, 4: 23, 5: 24, 6: 22, 7: 16, 8: 18, 9: 13, 10: 18, 11: 31, 12: 13, 13: 19, 14: 17, 15: 13, 16: 14, 17: 21, 18: 37, 19: 8, 20: 11, 21: 7, 22: 17, 23: 12, 24: 12, 25: 13, 26: 16, 27: 5, 28: 23, 29: 12, 30: 21, 31: 30, 32: 6, 33: 10, 34: 4, 35: 41, 36: 9, 37: 15, 38: 4, 39: 18, 40: 30, 41: 19, 42: 38, 43: 19, 44: 9, 45: 14, 46: 13, 47: 12, 48: 8, 49: 11, 50: 27, 51: 29}
group = 0
local_counter = 0
global_counter = 0
local counter = 1
never hit the stop spot
about to append 1 (array([371942.875   , 276732.78125 ,  21699.796875]), array([372336.84375   , 277893.        ,  23490.79296875]))
group = 1
local_counter = 15
global_counter = 15
local counter = 16
never hit the stop spot
about to append 1 (array([371942.875   , 276732.78125 ,  21699.796875]), array([373741.65625   , 276315.375     ,  23699.85351562]))
group = 2
local_counter = 29
global_counter = 29
local counter = 30
never hit the stop spot
about to append 1 (array([372336.84375   , 277893.        ,  23

In [275]:
#filter away the edges that are not within the threshold distance
""" pseudocode: 
Go through one edge group (designated by counter) at a time
1) If first point is under threshold --> put as "start" else...
2) go find first point under threshold and put the average of previous and that one in list "start"
    c. If never find one then just move onto next list
3) Go until you find under threshold: 
    a. Once do, add the average of previous and that one as stop and add (start,stop to list)
    b. if never find end then add last point as stop and add (start,stop to list)
4) Repeat step 4



1) if below the threshold, and new counter, add to list
2) if above threshold, and new counter --> ignore and move to next one
3) if below the threshold but not a new counter --> ignore and move on
4) if above threshold but not a new counter --> add it to the list as endpoint

*****Change to be made from last version *****
For Post-synaptic need to save off all of the discretized portions
For Presynaptic: save off the consolidated

"""
# boundary_points = [((np.linspace(d,discrete_edges_stacked[i+1],3))[1]) 
#                            for i,d in enumerate(discrete_edges_stacked) 
#                            if i <(len(discrete_edges_stacked) - 1)]

# boundary_points.insert(0,discrete_edges_stacked[0])
# boundary_points.append(discrete_edges_stacked[len(discrete_edges_stacked)-1])

start_time = time.time()

myCounter = Counter(discrete_edges_labels_stacked)
print(dict(myCounter.items()))

global_counter = 0
#lists to use for postsyn coordinates:  nearest_nodes,boundary_nearest_nodes 
final_postsyn_contact_skeleton = []
#lists to use for presyn coordinates:  discrete_edges_stacked, boundary_points
final_presyn_contact_skeleton = []

boundary_points 
for group,number_pts in dict(myCounter).items():
    #find first point that is under threshold
    local_counter = global_counter
    
    found_End = False
    while(found_End==False):
        #print("group = " + str(group))
        #print("local_counter = " + str(local_counter))
        #print("global_counter = " + str(global_counter))
        start_presyn = np.array([])
        start_postsyn = np.array([])
        for i in range(local_counter,global_counter + number_pts):
            if distances[i]<threshold:
                if i == global_counter:
                    #print(nearest_nodes[i])
                    #postsyn should be the nearest neighbor
                    start_postsyn=vertices_whole[nearest_nodes[i]]
                    #presyn should be the actual edge point
                    start_presyn=discrete_edges_stacked[i]
                    
                else:
                    start_postsyn=vertices_whole[boundary_nearest_nodes[i]]
                    start_presyn=boundary_points[i]
                local_counter = i + 1
                break

        #if got through whole list and not find start point
        if not start_postsyn.any():
            #print("went through whole list")
            global_counter += number_pts
            found_End = True
            break
        else: #if did get a start value 
            # go until you get an end point 
            #print("local counter = " + str(local_counter))
            stop_postsyn = np.array([])
            stop_presyn = np.array([])
            for i in range(local_counter,global_counter + number_pts):
                if distances[i]>threshold:
                    stop_postsyn=vertices_whole[boundary_nearest_nodes[i]]
                    stop_presyn=boundary_points[i]
                    local_counter = i + 1
                    #print("local counter after stop = " + str(local_counter))
                    break
                else:
                    #for the post-syn want all the values being saved off
                    stop_postsyn=vertices_whole[boundary_nearest_nodes[i]]
                    #print("about to append 3 " + str((start_postsyn,stop_postsyn)))
                    final_postsyn_contact_skeleton.append((start_postsyn,stop_postsyn))
                    start_postsyn=vertices_whole[boundary_nearest_nodes[i]]
                    stop_postsyn = np.array([])

            #if never hit stop point
            if not stop_presyn.any():
                #print("never hit the stop spot")
                #print(nearest_nodes[global_counter + number_pts - 1])
                stop_postsyn = vertices_whole[nearest_nodes[global_counter + number_pts - 1]]
                stop_presyn = discrete_edges_stacked[global_counter + number_pts - 1]
                #add the stop and start to the list
                #print("about to append 1 " + str((start_postsyn,stop_postsyn)))
                final_postsyn_contact_skeleton.append((start_postsyn,stop_postsyn))
                final_presyn_contact_skeleton.append((start_presyn,stop_presyn))
                global_counter += number_pts
                #go onto next line segment
                found_End=True
                break
            else:
                #add the stop label
                #print("about to append 2 " + str((start_postsyn,stop_postsyn)))
                final_postsyn_contact_skeleton.append((start_postsyn,stop_postsyn))
                final_presyn_contact_skeleton.append((start_presyn,stop_presyn))

            #continue with loop if can find another start and stop
            
final_postsyn_contact_skeleton = np.array(final_postsyn_contact_skeleton)
final_presyn_contact_skeleton = np.array(final_presyn_contact_skeleton)
print("final_postsyn_contact_skeleton.shape = " + str(final_postsyn_contact_skeleton.shape))
print("final_presyn_contact_skeleton.shape = " + str(final_presyn_contact_skeleton.shape))


    


{0: 15, 1: 14, 2: 12, 3: 15, 4: 23, 5: 24, 6: 22, 7: 16, 8: 18, 9: 13, 10: 18, 11: 31, 12: 13, 13: 19, 14: 17, 15: 13, 16: 14, 17: 21, 18: 37, 19: 8, 20: 11, 21: 7, 22: 17, 23: 12, 24: 12, 25: 13, 26: 16, 27: 5, 28: 23, 29: 12, 30: 21, 31: 30, 32: 6, 33: 10, 34: 4, 35: 41, 36: 9, 37: 15, 38: 4, 39: 18, 40: 30, 41: 19, 42: 38, 43: 19, 44: 9, 45: 14, 46: 13, 47: 12, 48: 8, 49: 11, 50: 27, 51: 29}
final_postsyn_contact_skeleton.shape = (678, 2, 3)
final_presyn_contact_skeleton.shape = (45, 2, 3)


In [276]:
print(f"Time for exporting = {time.time()-start_time}")

Time for exporting = 0.12136077880859375


In [ ]:
#see if can get faster using just array builders

#for the postsyn: just need all edges for


In [267]:
np.savetxt("./csv_tests/boundary_nearest_nodes.csv", vertices_whole[boundary_nearest_nodes], delimiter=",")
np.savetxt("./csv_tests/distances.csv", distances, delimiter=",") 
np.savetxt("./csv_tests/nearest_nodes.csv", vertices_whole[nearest_nodes], delimiter=",")
np.savetxt("./csv_tests/discrete_edges_labels_stacked.csv", discrete_edges_labels_stacked, delimiter=",")
np.savetxt("./csv_tests/discrete_edges_stacked.csv", discrete_edges_stacked, delimiter=",")
np.savetxt("./csv_tests/boundary_points.csv", boundary_points, delimiter=",")

In [230]:
final_postsyn_contact_skeleton #went ahead and checked that the array matches

array([[[371942.875     , 276732.78125   ,  21699.796875  ],
        [372336.84375   , 277893.        ,  23490.79296875]],

       [[371942.875     , 276732.78125   ,  21699.796875  ],
        [373741.65625   , 276315.375     ,  23699.85351562]],

       [[372336.84375   , 277893.        ,  23490.79296875],
        [372933.34375   , 278085.90625   ,  23454.6484375 ]],

       [[369170.28125   , 268539.53125   ,  17792.63671875],
        [370130.125     , 268567.90625   ,  16738.83789062]],

       [[363490.65625   , 272605.1875    ,  15230.25488281],
        [364894.9375    , 274130.40625   ,  15496.27050781]],

       [[371747.3125    , 268894.90625   ,  15739.55273438],
        [371916.03125   , 268886.40625   ,  15186.50390625]],

       [[364894.9375    , 274130.40625   ,  15496.27050781],
        [364894.9375    , 274130.40625   ,  15496.27050781]],

       [[368472.375     , 268253.8125    ,  16670.53710938],
        [369170.28125   , 268539.53125   ,  17792.63671875]],

       [

In [ ]:
final_presyn_contact_skeleton

In [270]:
print("final_postsyn_contact_skeleton.shape = " + str(final_postsyn_contact_skeleton.shape))
print("final_presyn_contact_skeleton.shape = " + str(final_presyn_contact_skeleton.shape))
print(discrete_edges_labels_stacked.shape)
#find out how many of the distance are below threshold:
d_below = [d for d in distances if d < threshold]
print(len(d_below))


final_postsyn_contact_skeleton.shape = (678, 2, 3)
final_presyn_contact_skeleton.shape = (45, 2, 3)
(878,)
678


In [235]:
#How to filter the post-synaptic skeleton to be rid of the soma
"""
1) get inner bounding box
2) Find all vertices inside of there
3) Delete all edges with any of those vertices in there

"""

#gets boundingn box
dir(mesh.convex_hull)
mesh.export(str(neuron_target)+"_regulr.off","off")
mesh.convex_hull.export(str(neuron_target)+"_convex_hull.off","off")

help(mesh.convex_hull.contains(skeleton_edges))
#get a list of all of the unique

Help on method contains in module trimesh.base:

contains(points) method of trimesh.base.Trimesh instance
    Given a set of points, determine whether or not they are inside the mesh.
    This raises an error if called on a non- watertight mesh.
    
    Parameters
    ---------
    points : (n, 3) float
      Points in cartesian space
    
    Returns
    ---------
    contains : (n, ) bool
      Whether or not each point is inside the mesh



In [238]:
#go and get vertices of skeleton
skeleton_edges = (ta3p100.FilteredSkeleton() & "segment_id=" + str(neuron_target)).fetch("edges")[0]
skeleton_edges

#get the filtered edges according to bounding box:


array([[[390503. , 258004. ,  53092. ],
        [390297. , 258083. ,  53343.4]],

       [[360268. , 247349. ,  34906.8],
        [362103. , 247974. ,  35768.2]],

       [[371456. , 298452. ,  23197.6],
        [371367. , 297628. ,  23062.2]],

       ...,

       [[372090. , 279077. ,  18538.1],
        [371983. , 280015. ,  18723.2]],

       [[372090. , 279077. ,  18538.1],
        [372325. , 276610. ,  18883. ]],

       [[373011. , 275048. ,  16765. ],
        [372325. , 276610. ,  18883. ]]])

In [247]:
#use the reshape method
skeleton_edges.shape
reshaped_skeleton_edges = skeleton_edges.reshape(skeleton_edges.shape[0]*2,3)

In [252]:

mesh.convex_hull.contains(reshaped_skeleton_edges)


ModuleNotFoundError: No module named 'rtree'

In [259]:
#get the bounding box by using trimesh
print((vertices_whole).shape)
print(triangles_whole.shape)

mesh = trimesh_io.Mesh(vertices=vertices_whole, faces=triangles_whole)
#mesh.bounding_box.vertices
#mesh.bounding_box.export("./bbox_whole_upright.off","off")

#get the maximum points of the vertices

#two highest points for the bounding box
min_bb = np.array(mesh.bounding_box.vertices).min(0)
max_bb = np.array(mesh.bounding_box.vertices).max(0)


# #want to get the edges that still persist on the presynaptic
# neuron_axon = 648518346349507130
# neuron_target = 648518346341393609

# segment_id = neuron_target

# segmentation = 2
# query_key_presyn=dict(segment_id=segment_id,segmentation=segmentation)


#get the filtered edges according to bounding box:
filtered_edges_postsyn = filter_edges_by_bounding_box(skeleton_edges,min_bb,max_bb)

print(skeleton_edges.shape)
print(filtered_edges_postsyn.shape)




(33499, 3)
(66166, 3)
Loops took 0.0068817138671875 sec
(1326, 2, 3)
(0,)


In [254]:
import time
def filter_edges_by_bounding_box(edges,max__bb_zone,min_bb_zone):
    #can just use bounding box function to get rid of any inside edges
    
    start_time = time.time()
    filtered_edges = list()
    for i,e in enumerate(edges):

    #     print(min(e[0][0],e[1][0]))
    #     print(max_bb_zone[0])
    #     print(min(e[0][0],e[1][0])>max_bb_zone[0])

        if min(e[0][0],e[1][0])>max_bb_zone[0]:
            continue

        if max(e[0][0],e[1][0])<min_bb_zone[0]:
            continue

        if min(e[0][1],e[1][1])>max_bb_zone[1]:
            continue

        if max(e[0][1],e[1][1])<min_bb_zone[1]:
            continue

        if min(e[0][2],e[1][2])>max_bb_zone[2]:
            continue

        if max(e[0][2],e[1][2])<min_bb_zone[2]:
            continue

        filtered_edges.append(e)


    print(f"Loops took {time.time()-start_time} sec")
    return np.array(filtered_edges)